# Lesson 5: Prompt Compression

In [ ]:
import custom_utils

# Data loading

In [ ]:
from datasets import load_dataset
import pandas as pd

dataset = load_dataset("MongoDB/airbnb_embeddings", streaming=True, split="train")
dataset = dataset.take(100)
dataset_df = pd.DataFrame(dataset)
dataset_df.head(5)

# Document modelling

In [ ]:
listings = custom_utils.process_records(dataset_df)

# Database creation and connection

In [ ]:
db, collection = custom_utils.connect_to_database()

# Data ingestion

In [ ]:
custom_utils.setup_vector_search_index_with_filter(collection=collection)

# Handling query

In [ ]:
from pydantic import BaseModel
from typing import Optional


class SearchResultItem(BaseModel):
    name: str
    accommodates: Optional[int] = None
    address: custom_utils.Address
    neighborhood_overview: Optional[str] = None
    notes: Optional[str] = None
    averageReviewScore: Optional[float] = None
    number_of_reviews: Optional[float] = None
    combinedScore: Optional[float] = None

# Boosting search result after vector search

In [ ]:
review_average_stage = {
    "$addFields": {
        "averageReviewScore": {
            "$divide": [
                {
                    "$add": [
                        "$review_scores.review_scores_accuracy",
                        "$review_scores.review_scores_cleanliness",
                        "$review_scores.review_scores_checkin",
                        "$review_scores.review_scores_communication",
                        "$review_scores.review_scores_location",
                        "$review_scores.review_scores_value",
                    ]
                },
                6  # Divide by the number of review score types to get the average
            ]
        },
        "reviewCountBoost": "$number_of_reviews"
    }
}

In [ ]:
weighting_stage = {
    "$addFields": {
        "combinedScore": {
            "$add": [
                {"$multiply": ["$averageReviewScore", 0.3]},
                {"$multiply": ["$reviewCountBoost", 0.7]}
            ]
        }
    }
}

In [ ]:
sorting_stage_sort = {
    "$sort": {"combinedScore": -1, }
}

In [ ]:
additional_stages = [review_average_stage, weighting_stage, sorting_stage_sort]

# Modified handling user query

In [ ]:
from IPython.display import display, HTML
import pprint

def handle_user_query(query, db, collection, stages=[], vector_index="vector_index_text"):
    get_knowledge = custom_utils.vector_search_with_filter(query, db, collection, stages, vector_index)

    if not get_knowledge:
        return "No results found.", "No source information available."
    
    search_results_models = [
        SearchResultItem(**result)
        for result in get_knowledge
    ]

    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    print("Uncompressed Prompt (Query Info):\n")
    print(search_results_df)

    completion = custom_utils.openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system", 
                "content": "You are a airbnb listing recommendation system."},
            {
                "role": "user", 
                "content": f"Answer this user query: {query} with the following context:\n{search_results_df}"
            }
        ]
    )
    system_response = completion.choices[0].message.content
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")
    display(HTML(search_results_df.to_html()))
    return system_response

In [ ]:
query = """
I want to stay in a place that's warm and friendly, 
and not too far from resturants, can you recommend a place? 
Include a reason as to why you've chosen your selection"
"""
handle_user_query(
    query, 
    db, 
    collection, 
    additional_stages, 
    vector_index="vector_index_with_filter"
)

# Prompt compression

In [ ]:
import json
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(
    model_name="microsoft/llmlingua-2-bert-base-multilingual-cased-meetingbank",
    model_config={"revision": "main"},
    use_llmlingua2=True,
    device_map="cpu",
)

def compress_query_prompt(query):
    
    demonstration_str=query['demonstration_str']
    instruction=query['instruction']
    question=query['question']
    
    compressed_prompt = llm_lingua.compress_prompt(
        demonstration_str.split("\n"), 
        instruction=instruction,
        question=question,
        target_token=500,
        rank_method="longllmlingua", 
        context_budget="+100",
        dynamic_context_compression_ratio=0.4,
        reorder_context="sort",
    )

    return json.dumps(compressed_prompt, indent=4)

In [ ]:
def handle_user_query_with_compression(query, db, collection, stages=[], vector_index="vector_index_text"):
    # Perform vector search to get knowledge from the database
    get_knowledge = custom_utils.vector_search_with_filter(query, db, collection, stages, vector_index)

    # Check if there are any results
    if not get_knowledge:
        return None, "No results found."

    # Convert search results into a list of SearchResultItem models
    search_results_models = [SearchResultItem(**result) for result in get_knowledge]

    # Convert search results into a DataFrame for better rendering
    search_results_df = pd.DataFrame([item.dict() for item in search_results_models])

    # Prepare information for compression
    query_info = {
        'demonstration_str': search_results_df.to_string(),  # Results from information retrieval process
        'instruction': "Write a high-quality answer for the given question using only the provided search results.",
        'question': query  # User query
    }

    # Compress the query prompt using predefined function
    compressed_prompt = compress_query_prompt(query_info)

    # Optional: Print compressed prompts for debugging
    print("Compressed Prompt:\n")
    pprint.pprint(compressed_prompt)
    print("\n" + "=" * 80 + "\n")

    return search_results_df, compressed_prompt



In [ ]:
def handle_system_response(query, compressed_prompt):
    # Generate system response using OpenAI's completion
    completion = custom_utils.openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are an Airbnb listing recommendation system."
            },
            {
                "role": "user",
                "content": f"Answer this user query: {query} with the following context:\n{compressed_prompt}"
            }
        ]
    )

    system_response = completion.choices[0].message.content

    # Print User Question, System Response
    print(f"- User Question:\n{query}\n")
    print(f"- System Response:\n{system_response}\n")

    # Return the system response
    return system_response


In [ ]:
# Compress the query and get search results
results, compressed_prompt = handle_user_query_with_compression(query, 
                                                                  db, 
                                                                  collection, 
                                                                  additional_stages, 
                                                                  vector_index="vector_index_with_filter"
                                                                 )

In [ ]:
if compressed_prompt:
    # Handle the system response with the compressed prompt
    system_response = handle_system_response(query, compressed_prompt)
else:
    print("No valid results to display.")